Lien du projet kaggle :
https://www.kaggle.com/code/jyotiprasadpal/project-building-a-conversational-chatbot/notebook

In [46]:
import numpy as np
import pandas as pd
import torch
import os
import re
import time

from torchtext.vocab import Vocab
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as transforms
import torch.nn as nn

from collections import Counter

In [2]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
device = torch.device(device)

mps


In [3]:
lines = open('./chatbot_data/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conv_lines = open('./chatbot_data/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [4]:
lines[100].split(' +++$+++ ')[:-1]

['L394', 'u4', 'm0', 'JOEY']

In [5]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')

    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [6]:
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [7]:
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    convs.append((_line.split((","))))

In [8]:
convs

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [9]:
questions = []
targets = []

for conv in convs:
    for i in range(len(conv) - 1):
        questions.append(id2line[conv[i]])
        targets.append(id2line[conv[i+1]])

print(len(questions))
print(len(targets))

221616
221616


In [10]:
def clean_text(text):
    text = text.lower()

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [11]:
clean_questions = []
clean_targets = []

for quest in questions:
    clean_questions.append(clean_text(quest))

for targ in targets:
    clean_targets.append(clean_text(targ))

In [12]:
clean_questions

['can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part please',
 'you are asking me out that is so cute that is your name again',
 'no no it is my fault we did not have a proper introduction',
 'cameron',
 'the thing is cameron i am at the mercy of a particularly hideous breed of loser my sister i cannot date until she does',
 'why',
 'unsolved mystery she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head",
 'right see you are ready for the quiz',
 'i do not want to know how to say that though i want to know useful things like where the good stores are how much does champagne cost stuff like chat i have never in my life had to point out my head to s

In [13]:
VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
MAX_LEN = 20
NUM_SAMPLES = 60000
GLOVE_DIR = './data//input/glove-global'
EPOCHS = 40
BATCH_SIZE = 256

In [14]:
for i, val in enumerate(zip(clean_questions, clean_targets)):
    print(i)
    print(val[0])
    print(val[1])
    break

0
can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again
well i thought we would start with pronunciation if that is okay with you


In [15]:
min_line_length = 2
max_line_length = 20

short_questions = []
short_answers = []

for i, val in enumerate(zip(clean_questions, clean_targets)):
    if (len(val[0].split()) >= min_line_length and len(val[0].split()) <= max_line_length) and (
            min_line_length <= len(val[1].split()) <= max_line_length):
        short_questions.append(val[0])
        short_answers.append(val[1])

print(len(short_questions))
print(len(short_answers))


138335
138335


In [16]:
input_text = short_questions[:NUM_SAMPLES]

In [17]:
def tagger(input):
    bos = "<BOS> "
    eos = " <EOS>"

    final_target = [bos + text + eos for text in input]
    return final_target

decoder_input_text = tagger(short_answers[:NUM_SAMPLES])

In [59]:
def vocab_creator(text_list, VOCAB_SIZE):

    def yield_tokens(text_list):
        for text in text_list:
            yield text.split()

    special_tokens = ["<pad>", "<unk>"]

    vocab = build_vocab_from_iterator(yield_tokens(text_list), specials=special_tokens, max_tokens=VOCAB_SIZE)

    word2idx = {word: idx for idx, word in enumerate(vocab.get_itos())}
    idx2word = {idx: word for idx, word in enumerate(vocab.get_itos())}

    return word2idx, idx2word


In [61]:
word2idx, idx2word = vocab_creator(text_list=input_text+decoder_input_text, VOCAB_SIZE=VOCAB_SIZE)

In [76]:
def text2seq(word2idx, idx2word, encoder_text, decoder_text):
    encoder_sequences = []
    decoder_sequences = []

    for text in encoder_text:
        subtext = []
        for word in text.split():
            if word in word2idx:
                subtext.append(word2idx[word])
            else:
                subtext.append(word2idx["<unk>"])
        encoder_sequences.append(subtext)

    for text in decoder_text:
        subtext = []
        for word in text.split():
            if word in word2idx:
                subtext.append(word2idx[word])
            else:
                subtext.append(word2idx["<unk>"])
        decoder_sequences.append(subtext)

    return encoder_sequences, decoder_sequences

In [77]:
test, po = text2seq(word2idx, idx2word, input_text, decoder_input_text)

In [78]:
po

[[2, 8, 6, 10498, 19, 8504, 19, 6770, 395, 140, 3],
 [2, 102, 87, 43, 38, 20, 222, 49, 84, 1108, 19977, 2023, 151, 3],
 [2, 289, 10, 3],
 [2, 123, 17, 70, 15, 5, 53, 12, 3],
 [2, 56, 70, 4, 14, 357, 27, 6, 11204, 3],
 [2, 289, 1108, 3],
 [2, 52, 120, 253, 5, 51, 212, 32, 3],
 [2, 4, 14, 659, 3],
 [2,
  1,
  819,
  6241,
  269,
  132,
  349,
  19,
  5,
  96,
  152,
  282,
  11,
  19030,
  297,
  6,
  1,
  18699,
  3],
 [2, 5, 101, 101, 101, 334, 58, 41, 5, 72, 8, 816, 30, 545, 552, 3],
 [2, 5, 22, 348, 33, 10, 41, 47, 98, 8, 498, 9, 32, 65, 27, 50, 3],
 [2,
  75,
  356,
  44,
  5,
  16,
  152,
  228,
  68,
  77,
  7,
  13,
  47,
  36,
  4648,
  154,
  2312,
  11,
  176,
  13,
  9256,
  3],
 [2, 5, 146, 20, 1114, 82, 1306, 7395, 3],
 [2, 5, 31, 3],
 [2, 4, 96, 234, 9, 58, 49, 37, 17931, 34, 4, 3],
 [2, 87, 13, 7, 44, 4, 104, 9, 77, 3],
 [2, 15, 933, 3],
 [2, 17, 26, 13697, 2042, 12413, 5, 22, 39, 1520, 298, 3],
 [2, 145, 153, 55, 5, 104, 9, 204, 63, 113, 392, 38, 29, 19668, 3],
 [2, 15, 